In [13]:
!pip3 install ktrain

In [14]:
!pip3 install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-0tgswea2
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-0tgswea2
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106829 sha256=8cad493ad9cac162e842ef23cf9b8a9efca24ea8fca13695f51678f6c7fd1942
  Stored in directory: /tmp/pip-ephem-wheel-cache-fkg9kulw/wheels/51/59/0a/0f48442b8d209583a4453580938d7ba2270aca40edacee6d45
Successfully built eli5


In [15]:
import numpy as np
np.random.seed(123)

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_path = "drive/My Drive/GSA Challenge/gsa_challenge/ai-ml-challenge-2020/"

First, we'll split the data into train, validation, and test.

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv(data_path + 'data/full_data.csv')

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_data, test_data, train_labels, test_labels = train_test_split(data['Clause Text'], data['Classification'], 
                                                                    test_size=0.1, 
                                                                    stratify = data['Classification'])

In [9]:
# Base model
np.mean(test_labels == 0)

0.8134517766497462

In [10]:
from sklearn.metrics import f1_score, brier_score_loss

In [11]:
f1_score(test_labels, np.zeros(len(test_labels)))

0.0

In [12]:
brier_score_loss(test_labels, [np.mean(test_labels == 0)]*len(test_labels))

0.544756048854647

## Ktrain

In [17]:
import ktrain
from ktrain import text

In [18]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [19]:
pd.concat([train_data, train_labels], axis=1)

,Clause Text,Classification
114,All payments by the Customer hereunder shall b...,0
4749,THIS END USER LICENSE AGREEMENT (the “Agreemen...,0
666,"1.5 Subject to the terms hereof, Company will ...",0
2245,(b) Multiple Sites/Services. If Customer has o...,0
7550,Note: Ordering Activities are only required to...,0
...,...,...
5498,"By executing this Agreement in writing, you ag...",0
926,REPRESENTATIONS AND WARRANTIES,0
2614,6.1 Warranties; Disclaimers. COMPANY warrants ...,0
961,FURTHER RESTRICTIONS ON USE FOR SOFTWARE WITH ...,1


In [20]:
pd.concat([test_data, test_labels], axis=1)

,Clause Text,Classification
5803,LOSS OF ANTICIPATED SAVINGS; OR,0
6586,4.1.9 To install replacement items that are co...,0
5203,1. Definitions.\n“Device” means any physical o...,0
3621,Service Level Agreement Claims.,0
1051,4. Reserved.,0
...,...,...
660,Customer acknowledges that Vendor may desire t...,1
3882,LFF Enclosure midplane,0
1630,Ordering Activities MUST register their COMPAN...,0
7831,COMPANY warrants that to its knowledge it eith...,0


In [21]:
trn, val, preproc =  text.texts_from_df(pd.concat([train_data, train_labels], axis=1), 
                                                           text_column='Clause Text', # name of column containing review text
                                                           label_columns=['Classification'],
                                                           maxlen=512, 
                                                           max_features=100000,
                                                           preprocess_mode='distilbert',
                                                           val_df = pd.concat([test_data, test_labels], axis=1),
                                                           ngram_range=1)

preprocessing train...
language: en
train sequence lengths:
	mean : 80
	95percentile : 262
	99percentile : 466


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 81
	95percentile : 265
	99percentile : 532


In [22]:
model = text.text_classifier('distilbert', trn, preproc=preproc)

Is Multi-Label? False
maxlen is 512



done.


In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6, workers=8)
learner.reset_weights()

/usr/local/lib/python3.6/dist-packages/ktrain/__init__.py:64: UserWarning: Changed use_multiprocessing to True because NumpyArrayIterator with workers>1 is slow when use_multiprocessing=False. If you experience issues with this, please set workers=1 and use_multiprocessing=False.
  I.warnings.warn(wrn_msg)


Model weights have been reset.


In [ ]:
# Train for THREE EPOCHS TOTAL

In [ ]:
# learner.lr_find(show_plot=True, max_epochs=2)

In [ ]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train on 7091 samples, validate on 788 samples
Epoch 1/3
7091/7091 [==============================] - 1298s 183ms/sample - loss: 0.4044 - accuracy: 0.8119 - val_loss: 0.3761 - val_accuracy: 0.8160
Epoch 2/3
7091/7091 [==============================] - 1271s 179ms/sample - loss: 0.3140 - accuracy: 0.8588 - val_loss: 0.3055 - val_accuracy: 0.8706
Epoch 3/3
7091/7091 [==============================] - 1273s 180ms/sample - loss: 0.1695 - accuracy: 0.9299 - val_loss: 0.3203 - val_accuracy: 0.8794


In [ ]:
learner.validate()

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       641
           1       0.73      0.56      0.63       147

    accuracy                           0.88       788
   macro avg       0.82      0.76      0.78       788
weighted avg       0.87      0.88      0.87       788



array([[611,  30],
       [ 65,  82]])

In [ ]:
bert_predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
bert_predictor.save(data_path + "distilbert_predictor")

In [ ]:
bert_predictor = ktrain.load_predictor(data_path + "distilbert_predictor", batch_size=6)

In [ ]:
test_data

5803                      LOSS OF ANTICIPATED SAVINGS; OR
6586    4.1.9 To install replacement items that are co...
5203    1. Definitions.\n“Device” means any physical o...
3621                      Service Level Agreement Claims.
1051                                         4. Reserved.
                              ...                        
660     Customer acknowledges that Vendor may desire t...
3882                               LFF Enclosure midplane
1630    Ordering Activities MUST register their COMPAN...
7831    COMPANY warrants that to its knowledge it eith...
4864    “Confidential Information” has the meaning asc...
Name: Clause Text, Length: 788, dtype: object

In [ ]:
import time

In [ ]:
start = time.time()
bertpreds_prob = bert_predictor.predict_proba(test_data.tolist())[:, 1]
bertpreds = [int(p > 0.5) for p in bertpreds_prob]
end = time.time()

print(f"Predicting entire test set (length {len(test_data)}) took {end - start} seconds")

Predicting entire test set (length 788) took 23.037274837493896 seconds


In [ ]:
import numpy as np 
np.mean(np.array(bertpreds) == test_labels)

0.8705583756345178

In [ ]:
from sklearn.metrics import f1_score, brier_score_loss, precision_score

In [ ]:
f1_score(test_labels, bertpreds, average='weighted')

0.8698669836794147

In [ ]:
brier_score_loss(test_labels, bertpreds_prob)

0.09073964408312715

In [ ]:
precision_score(test_labels, bertpreds, average='weighted')

0.8692361076141032

# Running on Given Validation Data

In [ ]:
gsa_challenge_valid = pd.read_csv(data_path + "data/AI_ML_Challenge_Validation_Data_Set_v1.csv")

In [ ]:
gsa_challenge_preds = bert_predictor.predict_proba(gsa_challenge_valid['Clause Text'].tolist())

In [ ]:
(gsa_challenge_preds[:, 1] > 0.5).astype(float)

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
final_preds_df = pd.DataFrame.from_dict({"Clause ID": gsa_challenge_valid['Clause ID'], 
                        "Prediction": (gsa_challenge_preds[:, 1] > 0.5).astype(float), 
                        "Probability Acceptable": gsa_challenge_preds[:, 0]})

In [ ]:
final_preds_df.to_csv(data_path + "Andrew and Rohan Validation Data File.csv", index=False)

# Explainability

In [ ]:
bert_predictor.explain(gsa_challenge_valid['Clause Text'].iloc[50])

In [ ]:
unacceptable_preds = test_data.iloc[np.where(np.asarray(bertpreds) == 1)[0]]

In [ ]:
unacceptable_preds

1276    THIS LICENSING AGREEMENT (“Agreement”) is ente...
2821    Increase to Number of Users.  Client acknowled...
3535    Company Request: \n•        Company Services I...
6208    Limitation on Liability.\n\nEXCEPT FOR THE COM...
4541    EULA and in accordance with Company’s attached...
                              ...                        
1684    Company shall invoice Customer shall pay to Co...
6023    The following four sentences only apply if You...
3098    The features of Company are set forth in the S...
1196    THE LICENSED PRODUCT IS NOT DESIGNED, MANUFACT...
90      3.1\tRights to the Platform. You acknowledge a...
Name: Clause Text, Length: 215, dtype: object

In [ ]:
for i, u in enumerate(np.random.choice(unacceptable_preds, size=10)):
  print(f'{i}: {u}')

0: [insert currency code] (excl. Taxes)
1: For Ordering Activity’s that are Federal Agencies, this Section shall not impair the Ordering Activity’s right to recover for personal injury or death resulting from Licensor’s negligence, or fraud or crimes arising out of or related to this agreement under any federal fraud statute, including the False Claims Act, 31 U.S.C. §§ 3729-3733.
2: Confidential Information” means (i) this Agreement and any other agreements between the Parties related to the subject matter hereof and the discussions, negotiations and proposals related to those agreements; (ii)  any information concerning the other Party’s business including, without limitation, the Company proprietary hardware and software platforms included in, but not limited to, the Company, all other software (including without limitation in object or source code form) and documentation and all other tangible, intangible, visual, electronic, written, oral, present, or future information such as a)

In [ ]:
most_confident_unacceptable = np.argsort(np.array(bertpreds_prob))[:10]
most_confident_acceptable = np.argsort(np.array(bertpreds_prob))[-10:]

In [ ]:
test_data.iloc[most_confident_acceptable]

2777                                      Data Protection
591                                          9. Reserved.
4913                                         9. Reserved.
6133                                        10. Reserved.
1170                                        10. Reserved.
208     1. Definitions and Interpretation\n1.1 Unless ...
2638                                        13. Reserved.
4493                                                   \n
1699                                        8.  Reserved.
6164                       4\tNo guarantee of resolution.
Name: Clause Text, dtype: object

In [ ]:
test_data.iloc[most_confident_unacceptable]

6268    This Agreement shall be governed by and constr...
7041    Governing Law, Jurisdiction, and Venue\nAll ma...
2482    Governing Law and Venue.  This Agreement and p...
3718    Except to the extent applicable law provides o...
128     This Agreement shall be governed by applicable...
5163    16.2.        In the event of a dispute between...
5128    11.\tGoverning Law and Venue. This Agreement s...
1827    The Company hereby grants you a limited, non-e...
6956    .  COSTS OF ARBITRATION.  Payment of all filin...
6982    This License shall be effective until terminat...
Name: Clause Text, dtype: object

In [ ]:
learner.view_top_losses(n=5, preproc=preproc)

----------
id:218 | loss:5.14 | true:Classification | pred:not_Classification)

----------
id:685 | loss:4.96 | true:Classification | pred:not_Classification)

----------
id:321 | loss:3.11 | true:Classification | pred:not_Classification)

----------
id:684 | loss:2.88 | true:Classification | pred:not_Classification)

----------
id:575 | loss:2.68 | true:Classification | pred:not_Classification)

